In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

Read in the J1939 Faults (Cummins' Connected Diagnostics file)

In [2]:
j1939faults = pd.read_csv('data/J1939Faults.csv', low_memory = False)
j1939faults.head()


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [3]:
j1939faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   RecordID               1187335 non-null  int64  
 1   ESS_Id                 1187335 non-null  int64  
 2   EventTimeStamp         1187335 non-null  object 
 3   eventDescription       1126490 non-null  object 
 4   actionDescription      0 non-null        float64
 5   ecuSoftwareVersion     891285 non-null   object 
 6   ecuSerialNumber        844318 non-null   object 
 7   ecuModel               1122577 non-null  object 
 8   ecuMake                1122577 non-null  object 
 9   ecuSource              1187335 non-null  int64  
 10  spn                    1187335 non-null  int64  
 11  fmi                    1187335 non-null  int64  
 12  active                 1187335 non-null  bool   
 13  activeTransitionCount  1187335 non-null  int64  
 14  faultValue        

In [4]:
j1939faults.ecuSerialNumber.nunique()

1989

In [5]:
j1939faults.ecuSource.unique()

array([ 0, 11, 49, 61,  3], dtype=int64)

In [6]:
j1939faults.ecuMake.unique()

array(['unknown', 'VOLVO', 'CMMNS', '?????', 'PCAR', nan, '?CAR', '?MMNS',
       '???R', '?????MX', '??MNS', 'BNDWS', 'PACCR', '?ACCR', '????S',
       '?NDWS', '????R', 'EATON', '?????MX16U13D13', '?ATON', '??DWS',
       '???CR', '5516014'], dtype=object)

In [7]:
#drop these columns because they are all Nan. (did not put 'axis=1')
j1939faults = j1939faults.drop(columns=['actionDescription', 'faultValue'])

In [8]:
#convert to lowercase 
j1939faults.columns=j1939faults.columns.str.lower()

In [9]:
j1939faults.head()

,recordid,ess_id,eventtimestamp,eventdescription,ecusoftwareversion,ecuserialnumber,ecumodel,ecumake,ecusource,spn,fmi,active,activetransitioncount,equipmentid,mctnumber,latitude,longitude,locationtimestamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,False,127,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [10]:
#convert dates/times to datetime format
j1939faults[['eventtimestamp', 'locationtimestamp']]=j1939faults[["eventtimestamp", "locationtimestamp"]].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S.%f')

In [11]:
#convert all columns except "active", which is boolean, to string
j1939faults[['recordid', 'ess_id', 'eventdescription', 'ecusoftwareversion', 'ecuserialnumber', 'ecumodel', 'ecusource', 'spn', 
            'fmi', 'activetransitioncount', 'equipmentid', 'mctnumber']] = j1939faults[['recordid', 'ess_id', 'eventdescription', 'ecusoftwareversion', 'ecuserialnumber', 'ecumodel', 'ecusource', 'spn', 
            'fmi', 'activetransitioncount', 'equipmentid', 'mctnumber']].astype("str")

In [12]:
j1939faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   recordid               1187335 non-null  object        
 1   ess_id                 1187335 non-null  object        
 2   eventtimestamp         1187335 non-null  datetime64[ns]
 3   eventdescription       1187335 non-null  object        
 4   ecusoftwareversion     1187335 non-null  object        
 5   ecuserialnumber        1187335 non-null  object        
 6   ecumodel               1187335 non-null  object        
 7   ecumake                1122577 non-null  object        
 8   ecusource              1187335 non-null  object        
 9   spn                    1187335 non-null  object        
 10  fmi                    1187335 non-null  object        
 11  active                 1187335 non-null  bool          
 12  activetransitioncount  11873

In [13]:
#finally, rename columns that need a _
j1939faults = j1939faults.rename(columns = {'recordid':'record_id', 'eventtimestamp':'event_timestamp', 'eventdescription':'event_descr', 'ecusoftwareversion':'ecu_software', 'ecuserialnumber': 'ecu_serial', 
                                            'ecumodel':'ecu_model', 'ecumake':'ecu_make', 'ecusource':'ecu_source', 'activetransitioncount':'active_trans_count', 'equipmentid':'equipment_id', 'mctnumber':'mct_number',
                                           'locationtimestamp':'location_timestamp'})

In [14]:
j1939faults.tail(3)

,record_id,ess_id,event_timestamp,event_descr,ecu_software,ecu_serial,ecu_model,ecu_make,ecu_source,spn,fmi,active,active_trans_count,equipment_id,mct_number,latitude,longitude,location_timestamp
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,nan,nan,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49
1187334,1248458,123906131,2020-03-06 14:15:34,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,nan,nan,NaN,0,111,18,False,8,2377,108605700,35.027314,-85.323472,2020-03-06 14:15:30


Read in the Vehicle Diagnostics Onboard Data file

In [15]:
veh_diag = pd.read_csv('data/VehicleDiagnosticOnboardData.csv')
veh_diag.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [16]:
veh_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12821626 entries, 0 to 12821625
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Id       int64 
 1   Name     object
 2   Value    object
 3   FaultId  int64 
dtypes: int64(2), object(2)
memory usage: 391.3+ MB


In [17]:
veh_diag.Name.value_counts()

LampStatus                   1187335
IgnStatus                     608454
EngineRpm                     586921
IntakeManifoldTemperature     586291
EngineOilPressure             586244
EngineCoolantTemperature      586071
BarometricPressure            585976
DistanceLtd                   585819
EngineLoad                    585621
FuelRate                      585237
FuelLtd                       585195
Speed                         583916
EngineOilTemperature          583912
TurboBoostPressure            583351
EngineTimeLtd                 581366
CruiseControlSetSpeed         576458
CruiseControlActive           574916
AcceleratorPedal              531889
FuelLevel                     502795
Throttle                      420503
ParkingBrake                  399972
FuelTemperature               299110
SwitchedBatteryVoltage        114059
ServiceDistance                  215
Name: Name, dtype: int64

In [18]:
diagnostics = veh_diag.pivot(index = 'FaultId', columns='Name', values='Value').reset_index()

In [19]:
diagnostics.tail()

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,863.25,586.75,23.6,4042.49282573,0,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74
1187334,1248458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#diagnostics = diagnostics.drop(columns=['Name'])

In [21]:
# change column name FaultID to record_id to merge with the j1939 dataset
diagnostics = diagnostics.rename(columns={'FaultId':'record_id'})

In [22]:
diagnostics.head(3)

Name,record_id,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 25 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   record_id                  1187335 non-null  int64 
 1   AcceleratorPedal           531889 non-null   object
 2   BarometricPressure         585976 non-null   object
 3   CruiseControlActive        574916 non-null   object
 4   CruiseControlSetSpeed      576458 non-null   object
 5   DistanceLtd                585819 non-null   object
 6   EngineCoolantTemperature   586071 non-null   object
 7   EngineLoad                 585621 non-null   object
 8   EngineOilPressure          586244 non-null   object
 9   EngineOilTemperature       583912 non-null   object
 10  EngineRpm                  586921 non-null   object
 11  EngineTimeLtd              581366 non-null   object
 12  FuelLevel                  502795 non-null   object
 13  FuelLtd                    

In [24]:
#convert record_id to string in order to merge
diagnostics[['record_id']] = diagnostics[['record_id']].astype("str")

Join the j1939faults with the diagnostics dataset

In [25]:
join_j1939_diag = j1939faults.merge(diagnostics, on = 'record_id', suffixes = ('_j1939', '_diag'))

In [26]:
join_j1939_diag.shape

(1187335, 42)

In [27]:
join_j1939_diag.head()

,record_id,ess_id,event_timestamp,event_descr,ecu_software,ecu_serial,ecu_model,ecu_make,ecu_source,spn,fmi,active,active_trans_count,equipment_id,mct_number,latitude,longitude,location_timestamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34,nan,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,False,127,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41,nan,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


Before more cleaning:  

Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)

Remove faults where the EquipmentID has more than 5 characters.

Remove faults where 'active' is set to False 

In [28]:
join_j1939_diag.columns

Index(['record_id', 'ess_id', 'event_timestamp', 'event_descr', 'ecu_software',
       'ecu_serial', 'ecu_model', 'ecu_make', 'ecu_source', 'spn', 'fmi',
       'active', 'active_trans_count', 'equipment_id', 'mct_number',
       'latitude', 'longitude', 'location_timestamp', 'AcceleratorPedal',
       'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed',
       'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad',
       'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure'],
      dtype='object')

In [29]:
join_j1939_diag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 0 to 1187334
Data columns (total 42 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   record_id                  1187335 non-null  object        
 1   ess_id                     1187335 non-null  object        
 2   event_timestamp            1187335 non-null  datetime64[ns]
 3   event_descr                1187335 non-null  object        
 4   ecu_software               1187335 non-null  object        
 5   ecu_serial                 1187335 non-null  object        
 6   ecu_model                  1187335 non-null  object        
 7   ecu_make                   1122577 non-null  object        
 8   ecu_source                 1187335 non-null  object        
 9   spn                        1187335 non-null  object        
 10  fmi                        1187335 non-null  object        
 11  active                     1187335 no

In [30]:
#delete all rows where lat/long is the service area.  To solve issue of "within vicinity", 
#shorten each lat/long to .0001 decimal points to broaden the coordinate zones 

# service_ctrs = join_j1939_diag[ 
#     (join_j1939_diag['latitude'] == 36.0666667) & (join_j1939_diag['longitude'] == -86.43472222) |
#     (join_j1939_diag['latitude'] == 35.5883333) & (join_j1939_diag['longitude'] == -86.4438888) |
#     (join_j1939_diag['latitude'] == 36.1950) & (join_j1939_diag['longitude'] == -83.174722)
#     ].index

#remove last 3 digits from each coordinate and make it >= lat & <= long
# service_ctrs = join_j1939_diag[ 
#     ((join_j1939_diag['latitude'] >= 36.0666) & (join_j1939_diag['longitude'] <= -86.4347)) |
#     ((join_j1939_diag['latitude'] >= 35.5883) & (join_j1939_diag['longitude'] <= -86.4438)) |
#     ((join_j1939_diag['latitude'] >= 36.1) & (join_j1939_diag['longitude'] <= -83.174))
#     ].index

#.0001 +/- swing on each lat long
# service_ctrs = join_j1939_diag[ 
#     ((join_j1939_diag['latitude'].between(36.0665667, 36.0667667, inclusive=True)) & (join_j1939_diag['longitude'].between(-86.43482222, -86.43462222, inclusive=True))) |
#     ((join_j1939_diag['latitude'].between(35.5882333, 35.5884333, inclusive=True)) & (join_j1939_diag['longitude'].between(-86.4439888, -86.4437888, inclusive=True))) |
#     ((join_j1939_diag['latitude'].between(36.1949, 36.1951, inclusive=True)) & (join_j1939_diag['longitude'].between(-83.174822, -83.174622, inclusive=True)))
#     ].index

#.001 +/- swing on each lat long
# service_ctrs = join_j1939_diag[ 
#     ((join_j1939_diag['latitude'].between(36.0656667, 36.0676667, inclusive=True)) & (join_j1939_diag['longitude'].between(-86.4357222, -86.4337222, inclusive=True))) |
#     ((join_j1939_diag['latitude'].between(35.5873333, 35.5893333, inclusive=True)) & (join_j1939_diag['longitude'].between(-86.4448888, -86.4428888, inclusive=True))) |
#     ((join_j1939_diag['latitude'].between(36.194, 36.196, inclusive=True)) & (join_j1939_diag['longitude'].between(-83.175722, -83.173722, inclusive=True)))
#     ].index

#.01 +/- swing and truncate on each lat long. **NOTE**: I increased the spread for the last lat (18-21 vs 19-20)
service_ctrs = join_j1939_diag[ 
    ((join_j1939_diag['latitude'].between(36.05, 36.07, inclusive=True)) & (join_j1939_diag['longitude'].between(-86.44, -86.42, inclusive=True))) |
    ((join_j1939_diag['latitude'].between(35.57, 35.59, inclusive=True)) & (join_j1939_diag['longitude'].between(-86.45, -86.43, inclusive=True))) |
    ((join_j1939_diag['latitude'].between(36.18, 36.21, inclusive=True)) & (join_j1939_diag['longitude'].between(-83.18, -83.16, inclusive=True)))
    ].index

In [31]:
type(service_ctrs)
#test

pandas.core.indexes.numeric.Int64Index

In [32]:
join_j1939_diag = join_j1939_diag.drop(service_ctrs, axis=0)


In [33]:
#type(join_j1939_diag)

In [34]:
join_j1939_diag.shape
# 1187260 rows when using full original lat/long coordinates
# using swing version of +/- (.0001) changed row count to 1,175,137 
# changing swing version to +/- .001 changed row count to 1,083,798
# changing swing version to +/- .01 and truncating lat/lon to .xx spaces, row count at 1,057,486


(1057486, 42)

In [35]:
#join_j1939_diag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057486 entries, 0 to 1187334
Data columns (total 42 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   record_id                  1057486 non-null  object        
 1   ess_id                     1057486 non-null  object        
 2   event_timestamp            1057486 non-null  datetime64[ns]
 3   event_descr                1057486 non-null  object        
 4   ecu_software               1057486 non-null  object        
 5   ecu_serial                 1057486 non-null  object        
 6   ecu_model                  1057486 non-null  object        
 7   ecu_make                   1001448 non-null  object        
 8   ecu_source                 1057486 non-null  object        
 9   spn                        1057486 non-null  object        
 10  fmi                        1057486 non-null  object        
 11  active                     1057486 no

In [36]:
#remove rows where equipment_id has 5 or more characters
join_j1939_diag = join_j1939_diag[join_j1939_diag['equipment_id'].map(len) < 6]

In [37]:
join_j1939_diag.shape

(1055687, 42)

In [38]:
#Remove rows where 'active' = False
join_j1939_diag = join_j1939_diag[join_j1939_diag['active'] == True]


In [39]:
join_j1939_diag.shape

(547766, 42)

In [40]:
join_j1939_diag.tail()

,record_id,ess_id,event_timestamp,event_descr,ecu_software,ecu_serial,ecu_model,ecu_make,ecu_source,spn,fmi,active,active_trans_count,equipment_id,mct_number,latitude,longitude,location_timestamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
1187324,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),nan,nan,CECU3B-NAMUX4,PACCR,49,96,3,True,126,1936,105355619,30.376851,-81.744953,2020-03-06 13:29:33,0,14.645,False,66.48672,391932.6,181.4,11,22.62,197.6,597.375,8016.75,60,51466.131256666,0.6208063,NaN,True,120.2,1279,False,NaN,0.9417657,NaN,100,1.16
1187328,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,True,93,1886,105351219,39.015694,-77.031157,2020-03-06 13:43:24,0,14.355,False,66.48672,457529.7,181.4,11,19.72,207.2188,600.25,13047.05,62,64491.926796682,0.5151371,NaN,True,104,2047,False,NaN,5.932153,NaN,100,0.58
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,nan,nan,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,863.25,586.75,23.6,4042.49282573,0,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [41]:
derate1569 = join_j1939_diag[join_j1939_diag['spn']=='1569']
derate1569.head()

,record_id,ess_id,event_timestamp,event_descr,ecu_software,ecu_serial,ecu_model,ecu_make,ecu_source,spn,fmi,active,active_trans_count,equipment_id,mct_number,latitude,longitude,location_timestamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
40,41,990856,2015-02-21 12:06:22,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57,0,14.21,False,66.48672,121095.5,174.2,0,35.96,220.4375,1048.125,2319.65,54.4,15620.097176682,0,32,True,51.8,18431,False,NaN,51.97187,3276.75,0,0.58
290,291,994045,2015-02-21 15:35:45,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,6,1721,105439606,37.735185,-85.808101,2015-02-21 15:36:21,76.4,14.2825,False,66.48672,121233.4,181.4,77,39.44,221,1561.125,2322.35,57.6,15638.32504827,13.24827,32,True,91.4,18431,False,NaN,40.73865,3276.75,0,15.66
340,341,994680,2015-02-21 16:22:24,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,7,1721,105439606,37.166666,-85.964027,2015-02-21 16:23:00,20.4,14.2825,False,66.48672,121274.7,181.4,0,37.7,213.575,1122.375,2323.15,55.6,15643.476403284,0.02641729,32,True,60.8,18431,False,NaN,55.39912,3276.75,0,0.87
378,379,995223,2015-02-21 17:08:02,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,10,1721,105439606,36.770324,-86.482870,2015-02-21 17:08:37,63.2,14.355,False,66.48672,121319.2,177.8,55,38.28,218.75,1274.125,2323.9,54,15649.420274454,6.960955,32,True,53.6,18431,False,NaN,62.89442,3276.75,0,2.32
1580,1581,1001939,2015-02-22 11:14:23,Condition Exists Engine Protection Torque Derate,unknown,unknown,unknown,unknown,0,1569,31,True,1,1515,105381937,30.376435,-83.299444,2015-02-22 11:14:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,18431,NaN,NaN,NaN,NaN,NaN,NaN


View 5246 codes (derates) for any trends 

In [42]:
derate5246 = join_j1939_diag[join_j1939_diag.spn == '5246']
derate5246.head(7)

,record_id,ess_id,event_timestamp,event_descr,ecu_software,ecu_serial,ecu_model,ecu_make,ecu_source,spn,fmi,active,active_trans_count,equipment_id,mct_number,latitude,longitude,location_timestamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
2089,2090,1011009,2015-02-23 05:05:44,nan,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902374,NaN,NaN,False,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN
2971,2972,1026305,2015-02-23 15:54:22,nan,unknown,unknown,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN
5713,5714,1070646,2015-02-25 13:53:08,nan,unknown,unknown,unknown,unknown,0,5246,0,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN
6534,6535,1097942,2015-02-26 22:24:29,nan,04993120*00021657*082113134117*07700053*I0*BBZ*,79466573,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1419,105355995,37.596805,-85.865555,2015-02-26 22:25:05,NaN,14.5,False,64.6226,441699.6,185,10,20.3,198.1625,648.125,9087.95,57.2,69605.769379298,0.6340149,NaN,True,140,22527,NaN,NaN,0,3276.75,NaN,0.58
6628,6629,1104361,2015-02-27 09:09:56,nan,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,40.534259,-76.431805,2015-02-27 09:10:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN
6665,6666,1109134,2015-02-27 12:45:34,nan,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,41.225879,-77.074907,2015-02-27 12:46:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN
6684,6685,1114036,2015-02-27 16:52:12,nan,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,41.033333,-77.515648,2015-02-27 16:52:49,0,14.5,False,64.6226,413001.4,165.2,12,25.52,170.4312,649.375,9368.75,74.8,63017.054230366,0.7661014,NaN,True,86,22527,NaN,NaN,0,3276.75,NaN,0.58


In [43]:
derate5246.shape
#using .001 lat long resulted in 503 rows, and .0001 resulted in 594 rows.  The .0001 yielded 
#lat long that yielded back Big G Express shop  


(496, 42)

In [44]:
#derate5246.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 2089 to 1181996
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   record_id                  496 non-null    object        
 1   ess_id                     496 non-null    object        
 2   event_timestamp            496 non-null    datetime64[ns]
 3   event_descr                496 non-null    object        
 4   ecu_software               496 non-null    object        
 5   ecu_serial                 496 non-null    object        
 6   ecu_model                  496 non-null    object        
 7   ecu_make                   413 non-null    object        
 8   ecu_source                 496 non-null    object        
 9   spn                        496 non-null    object        
 10  fmi                        496 non-null    object        
 11  active                     496 non-null    bool          
 12  a

In [45]:
# can drop column "ServiceDistance"
derate5246.ServiceDistance.unique()

array([nan], dtype=object)

Make sure Big G Express shops are not still showing up in report (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)

In [46]:
#derate5246.latitude.unique()

In [47]:
derate5246.event_descr.unique()

array(['nan'], dtype=object)

In [48]:
derate1569_5246 = join_j1939_diag[
    (join_j1939_diag['spn']=='1569') |
    (join_j1939_diag['spn']=='5246')
]
     
derate1569_5246.head(5)

,record_id,ess_id,event_timestamp,event_descr,ecu_software,ecu_serial,ecu_model,ecu_make,ecu_source,spn,fmi,active,active_trans_count,equipment_id,mct_number,latitude,longitude,location_timestamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
40,41,990856,2015-02-21 12:06:22,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57,0,14.21,False,66.48672,121095.5,174.2,0,35.96,220.4375,1048.125,2319.65,54.4,15620.097176682,0,32,True,51.8,18431,False,NaN,51.97187,3276.75,0,0.58
290,291,994045,2015-02-21 15:35:45,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,6,1721,105439606,37.735185,-85.808101,2015-02-21 15:36:21,76.4,14.2825,False,66.48672,121233.4,181.4,77,39.44,221,1561.125,2322.35,57.6,15638.32504827,13.24827,32,True,91.4,18431,False,NaN,40.73865,3276.75,0,15.66
340,341,994680,2015-02-21 16:22:24,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,7,1721,105439606,37.166666,-85.964027,2015-02-21 16:23:00,20.4,14.2825,False,66.48672,121274.7,181.4,0,37.7,213.575,1122.375,2323.15,55.6,15643.476403284,0.02641729,32,True,60.8,18431,False,NaN,55.39912,3276.75,0,0.87
378,379,995223,2015-02-21 17:08:02,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,10,1721,105439606,36.770324,-86.482870,2015-02-21 17:08:37,63.2,14.355,False,66.48672,121319.2,177.8,55,38.28,218.75,1274.125,2323.9,54,15649.420274454,6.960955,32,True,53.6,18431,False,NaN,62.89442,3276.75,0,2.32
1580,1581,1001939,2015-02-22 11:14:23,Condition Exists Engine Protection Torque Derate,unknown,unknown,unknown,unknown,0,1569,31,True,1,1515,105381937,30.376435,-83.299444,2015-02-22 11:14:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,18431,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
derate1569_5246.equipment_id.sort_values(ascending=False)

48291      310
115565     310
129562     310
853540     310
401648     310
          ... 
83425     1339
82873     1339
5714      1329
5713      1329
9897      1328
Name: equipment_id, Length: 5548, dtype: object

In [50]:
derate1569_5246.groupby('spn')['equipment_id'].value_counts()

spn   equipment_id
1569  1490            190
      1692            139
      1505            133
      1445             99
      1444             95
                     ... 
5246  2021              1
      2109              1
      301               1
      304               1
      306               1
Name: equipment_id, Length: 682, dtype: int64

Read in the Service Fault Codes file

In [51]:
svc_fault_code = pd.read_csv('data/Service Fault Codes_1_0_0_167.csv')
svc_fault_code.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


In [52]:
svc_fault_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7124 entries, 0 to 7123
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Published in CES 14602  7124 non-null   object
 1   Cummins Fault Code      7124 non-null   int64 
 2   Revision                7124 non-null   int64 
 3   PID                     7124 non-null   object
 4   SID                     7124 non-null   object
 5   MID                     7124 non-null   object
 6   J1587 FMI               7124 non-null   int64 
 7   SPN                     7124 non-null   int64 
 8   J1939 FMI               7124 non-null   int64 
 9   J2012 Pcode             7124 non-null   object
 10  Lamp Color              7124 non-null   object
 11  Lamp Device             7124 non-null   object
 12  Cummins Description     7124 non-null   object
 13  Algorithm Description   2005 non-null   object
dtypes: int64(5), object(9)
memory usage: 779.3+ KB


In [53]:
svc_fault_code[svc_fault_code['J1939 FMI']==31]

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
79,Y,211,167,Not Mapped,216,0,11,1484,31,P0700,None,None,Additional Auxiliary Diagnostic Codes Logged -...,Auxiliary diagnostic codes are present in the ...
153,Y,299,167,Not Mapped,117,128,11,1384,31,Not Mapped,Amber,Warning,SAE J1939 Engine Commanded Shutdown - Conditio...,The engine has been commanded shutdown.
194,Y,359,167,Not Mapped,124,128,11,1664,31,Not Mapped,Red,Stop / Shutdown,Engine Failed Automatic Start - Condition Exists,The engine has failed to start.
201,Y,371,167,Not Mapped,126,128,11,1077,31,Not Mapped,Amber,Warning,"Fuel Pump Control Module, Static Timing Error ...",The fuel pump control module has experienced a...
255,Y,446,167,Not Mapped,151,0,11,611,31,Not Mapped,Amber,Warning,GSP Communication Failure - Condition Exists,There is a GSP communication failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


In [54]:
#join_j1939_diag = join_j1939_diag.drop((service_ctrs, inplace=True) & (join_j1939_diag[join_j1939_diag['equipment_id'].map(len) > 5]) & (join_j1939_diag[join_j1939_diag['active'] == False]))

Reference for dealing with NaNs and logistic regression, https://www.kaggle.com/cemsarier/preprocessing-filling-nan-and-logistic-regression
